In [1]:
import pandas as pd
import csv
import os
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
Working_dir = os.getcwd()

In [38]:
res = pd.read_excel( os.path.join( Working_dir, "Transformer load profiles.xlsx" ),skiprows=2 )
res.drop(res.columns[0],axis=1,inplace=True)

In [44]:
res.columns.values[0]='hour'

res.head()

,hour,0,0.1,0.2,0.3,0.4
0,1,17.988825,17.988825,17.988825,17.988825,17.988825
1,2,17.854960,17.854960,17.854960,17.854960,17.854960
2,3,15.955075,15.955075,15.955075,15.955075,15.955075
3,4,16.282195,16.282195,16.282195,16.282195,16.282195
4,5,13.489930,13.489930,13.489930,13.489930,13.489930


-----------
Data analysis: when generation rate is different for columns

Negative generation rates mean that PV needed power itself?

Anyway, they will not affect the result since a formula below contains squares

In [67]:
print(res[res.ix[:,1]!=res.ix[:,2]])

      hour          0        0.1        0.2        0.3        0.4
1426  1427  14.622412  14.617737  14.613062  14.608387  14.603713
1427  1428  15.481214  15.476482  15.471750  15.467018  15.462286
1428  1429  15.421773  15.415007  15.408241  15.401474  15.394708
1429  1430  17.314813  17.311129  17.307444  17.303759  17.300075
1430  1431  16.096731  16.091272  16.085814  16.080355  16.074896
1431  1432  16.331657  16.324400  16.317142  16.309885  16.302628
1432  1433  15.465663  15.431731  15.397801  15.363873  15.329948
1433  1434  22.346143  22.344896  22.343648  22.342401  22.341154
1448  1449  17.265200  17.262655  17.260110  17.257566  17.255021
1449  1450   6.616222   6.566268   6.516319   6.466375   6.416436
1450  1451   7.750975   7.306732   6.862900   6.419478   5.976465
1451  1452  12.605895  11.758168  10.911933  10.067183   9.223914
1452  1453  26.796798  25.685652  24.577043  23.470961  22.367396
1453  1454  23.710359  22.468102  21.229020  19.993098  18.760324
1454  1455

------
Hours in a year (TH)

In [45]:
TH = res.iloc[:,0].max()
print("TH = ", TH)

TH =  8760


--------
Peak load of average loads

In [43]:
peak = res.iloc[:,1].max()
print("Peak avg load (kW) = ", peak)

Peak avg load (kW) =  36.1969245143


-----------
Annual no load loss (kWh) = No load loss (kW) * TH (h)

Statistics taken for a three–phase dry-type medium power transformer

No load loss = 280 W

In [46]:
no_load_loss = 0.28 * TH
print("Annual no load loss (kWh) = ", no_load_loss)

Annual no load loss (kWh) =  2452.8


------------
Annual load loss (kWh) = Load loss (kW) \* (load percent/100) \* TH (h) * LLF

$LLF = \sum_{n=1}^{TH} \frac{(Load_n)^2}{(Peak_load)^2} / TH$

Load loss = 2.05 kW from the requirements document

In [69]:
LLF_10 = sum( res.ix[:,1].apply(lambda x: x**2/peak**2 ))/TH
print( "LLF (10%) = ", LLF_10)

LLF_20 = sum( res.ix[:,2].apply(lambda x: x**2/peak**2 ))/TH
print( "LLF (20%) = ", LLF_20)

LLF_30 = sum( res.ix[:,3].apply(lambda x: x**2/peak**2 ))/TH
print( "LLF (30%) = ", LLF_30)

LLF_40 = sum( res.ix[:,4].apply(lambda x: x**2/peak**2 ))/TH
print( "LLF (40%) = ", LLF_40)

LLF (10%) =  0.166883028699
LLF (20%) =  0.154372050224
LLF (30%) =  0.154973470451
LLF (40%) =  0.16805669561


-------
Annual load loss (kWh) for 10% load

In [ ]:
loss = 2.05
load_loss_10 = loss * 0.1 * TH #* LLF_10
load_loss_20 = loss * 0.2 * TH #* LLF_20
load_loss_30 = loss * 0.3 * TH #* LLF_30
load_loss_40 = loss * 0.4 * TH #* LLF_40
res.ix[res.shape[0]]  = ["loss", no_load_loss, load_loss_10, load_loss_20, load_loss_30, load_loss_40]

In [93]:
print("Losses (kWh)\nno load: ", no_load_loss,"\n10% load: ", load_loss_10)
print("20% load: ", load_loss_20)
print("30% load: ", load_loss_30)
print("40% load: ", load_loss_40)

Losses (kWh)
no load:  2452.8 
10% load:  1795.8
20% load:  3591.6
30% load:  5387.4
40% load:  7183.2
